In [9]:
from logicqubit.logic import *
from cmath import *
import numpy as np
import sympy as sp
import scipy
from scipy.optimize import *
import matplotlib.pyplot as plt

See https://arxiv.org/abs/1304.3061

In [10]:
gates = Gates(1)

ID = gates.ID()
X = gates.X()
Y = gates.Y()
Z = gates.Z()

In [11]:
II = ID.kron(ID)

XX = X.kron(X)
XI = X.kron(ID)
IX = ID.kron(X)
XY = X.kron(Y)
XZ = X.kron(Z)

YY = Y.kron(Y)
YI = Y.kron(ID)
IY = ID.kron(Y)
YX = Y.kron(X)
YZ = Y.kron(Z)

ZZ = Z.kron(Z)
ZI = Z.kron(ID)
IZ = ID.kron(Z)
ZX = Z.kron(X)
ZY = Z.kron(Y)

sig_yy = [YY.get()[i,i] for i in range(len(YY.get()))]
sig_iz = [IZ.get()[i,i] for i in range(len(IZ.get()))]
sig_zi = [ZI.get()[i,i] for i in range(len(ZI.get()))]
sig_iz
ZZ.get()

array([[ 1,  0,  0,  0],
       [ 0, -1,  0,  0],
       [ 0,  0, -1,  0],
       [ 0,  0,  0,  1]])

In [12]:
H = XZ + ZI*2 # -ZX + 2*ZI

min(scipy.linalg.eig(H.get())[0])

(-2.23606797749979+0j)

In [13]:
def ansatz(reg, params):
    n_qubits = len(reg)
    depth = n_qubits
    for i in range(depth):
        for j in range(n_qubits):
            if(j < n_qubits-1):
                reg[j+1].CNOT(reg[j])
            reg[i].RY(params[j])
            
def ansatz_2q(q1, q2, params):
    q2.CNOT(q1)
    q1.RY(params[0])
    q2.RY(params[1])
    q1.CNOT(q2)
    q1.RY(params[2])
    q2.RY(params[3])
    q2.CNOT(q1)
    q1.RY(params[4])
    q2.RY(params[5])

In [14]:
def expectation_2q(params):
    logicQuBit  = LogicQuBit(2)
    q1 = Qubit()
    q2 = Qubit()

    ansatz_2q(q1,q2,params)
    psi = logicQuBit.getPsi()
    
    return (psi.adjoint()*H*psi).get()[0][0]

minimum = minimize(expectation_2q, [0,0,0,0,0,0], method='Nelder-Mead', options={'xtol': 1e-10, 'ftol': 1e-10})
print(minimum)

/home/cleoner/anaconda3/lib/python3.8/site-packages/scipy/optimize/optimize.py:586: ComplexWarning: Casting complex values to real discards the imaginary part
  fsim[k] = func(sim[k])
/home/cleoner/anaconda3/lib/python3.8/site-packages/scipy/optimize/optimize.py:611: ComplexWarning: Casting complex values to real discards the imaginary part
  fsim[-1] = fxe
/home/cleoner/anaconda3/lib/python3.8/site-packages/scipy/optimize/optimize.py:618: ComplexWarning: Casting complex values to real discards the imaginary part
  fsim[-1] = fxr
/home/cleoner/anaconda3/lib/python3.8/site-packages/scipy/optimize/optimize.py:614: ComplexWarning: Casting complex values to real discards the imaginary part
  fsim[-1] = fxr
/home/cleoner/anaconda3/lib/python3.8/site-packages/scipy/optimize/optimize.py:627: ComplexWarning: Casting complex values to real discards the imaginary part
  fsim[-1] = fxc
/home/cleoner/anaconda3/lib/python3.8/site-packages/scipy/optimize/optimize.py:637: ComplexWarning: Casting comp

 final_simplex: (array([[ 3.988717  ,  1.21161093, -0.99924905, -1.78975361, -0.80799974,
        -1.84695884],
       [ 3.988717  ,  1.21161093, -0.99924905, -1.78975361, -0.80799974,
        -1.84695884],
       [ 3.988717  ,  1.21161093, -0.99924905, -1.78975361, -0.80799974,
        -1.84695884],
       [ 3.988717  ,  1.21161093, -0.99924905, -1.78975361, -0.80799974,
        -1.84695884],
       [ 3.988717  ,  1.21161093, -0.99924905, -1.78975361, -0.80799974,
        -1.84695884],
       [ 3.988717  ,  1.21161093, -0.99924905, -1.78975361, -0.80799974,
        -1.84695884],
       [ 3.988717  ,  1.21161093, -0.99924905, -1.78975361, -0.80799974,
        -1.84695884]]), array([-2.23606798, -2.23606798, -2.23606798, -2.23606798, -2.23606798,
       -2.23606798, -2.23606798]))
           fun: -2.2360679774997907
       message: 'Optimization terminated successfully.'
          nfev: 650
           nit: 348
        status: 0
       success: True
             x: array([ 3.988717  ,  1

/home/cleoner/anaconda3/lib/python3.8/site-packages/scipy/optimize/optimize.py:644: ComplexWarning: Casting complex values to real discards the imaginary part
  fsim[j] = func(sim[j])


In [33]:
def expectation_value(measurements, base = np.array([1,-1,-1,1])):
    probabilities = np.array(measurements)
    expectation = np.sum(base * probabilities)
    return expectation

def sigma_xx(params):
    logicQuBit  = LogicQuBit(2, first_left = False)
    q1 = Qubit()
    q2 = Qubit()
    
    ansatz_2q(q1,q2,params)
    
    # medidas em XX
    q1.RY(-pi/2)
    q2.RY(-pi/2)
    
    result = logicQuBit.Measure([q1,q2])
    result = expectation_value(result)
    return result

def sigma_yy(params):
    logicQuBit  = LogicQuBit(2, first_left = False)
    q1 = Qubit()
    q2 = Qubit()
    
    ansatz_2q(q1,q2,params)
    
    # medidas em YY
    q1.RX(pi/2)
    q2.RX(pi/2)
    
    result = logicQuBit.Measure([q1,q2])
    result = expectation_value(result)
    return result

def sigma_zz(params):
    logicQuBit  = LogicQuBit(2, first_left = False)
    q1 = Qubit()
    q2 = Qubit()
    
    ansatz_2q(q1,q2,params)
          
    result = logicQuBit.Measure([q1,q2])
    zz = expectation_value(result)
    iz = expectation_value(result, sig_iz) # [zz, iz] = 0
    zi = expectation_value(result, sig_zi) # [zz, zi] = 0
    return zz, iz, zi

def sigma_zx(params):
    logicQuBit  = LogicQuBit(2, first_left = False)
    q1 = Qubit()
    q2 = Qubit()
    
    ansatz_2q(q1,q2,params)
    
    # medidas em X
    #q1.RY(-pi/2)
    q2.RY(-pi/2)
    
    result = logicQuBit.Measure([q1,q2])
    zx = expectation_value(result)
    return zx

def expectation_energy(params):
    #xx =  sigma_xx(params)
    #yy =  sigma_yy(params)
    zz, iz, zi = sigma_zz(params)
    zx = sigma_zx(params)

    result = -zx + zi*2
    return result

In [34]:
minimum = minimize(expectation_energy, [0,0,0,0,0,0], method='Nelder-Mead', options={'xtol': 1e-10, 'ftol': 1e-10})
print(minimum)

 final_simplex: (array([[ 0.37729988, -0.32129798,  0.33452791,  2.38985834, -1.3110545 ,
         0.47596158],
       [ 0.37729988, -0.32129798,  0.3345279 ,  2.38985834, -1.3110545 ,
         0.47596158],
       [ 0.37729988, -0.32129798,  0.3345279 ,  2.38985834, -1.3110545 ,
         0.47596158],
       [ 0.37729988, -0.32129798,  0.3345279 ,  2.38985834, -1.3110545 ,
         0.47596158],
       [ 0.37729988, -0.32129798,  0.3345279 ,  2.38985834, -1.3110545 ,
         0.47596158],
       [ 0.37729988, -0.32129798,  0.3345279 ,  2.38985834, -1.3110545 ,
         0.47596158],
       [ 0.37729988, -0.32129798,  0.3345279 ,  2.38985834, -1.3110545 ,
         0.47596158]]), array([-2.23606798, -2.23606798, -2.23606798, -2.23606798, -2.23606798,
       -2.23606798, -2.23606798]))
           fun: -2.2360679774997907
       message: 'Optimization terminated successfully.'
          nfev: 591
           nit: 313
        status: 0
       success: True
             x: array([ 0.37729988, -0

In [25]:
def gradient(params, evaluate):
    n_params = params.shape[0]
    shift = pi/2
    gradients = np.zeros(n_params)
    
    for i in range(n_params):
        #parameter shift rule
        shift_vect = np.array([shift if j==i else 0 for j in range(n_params)])
        shift_right = params + shift_vect
        shift_left = params - shift_vect
        
        expectation_right = evaluate(shift_right)
        expectation_left = evaluate(shift_left)

        gradients[i] = expectation_right - expectation_left

    return gradients

In [26]:
params = np.random.uniform(-np.pi, np.pi, 6)
last_params = np.zeros(6)

In [27]:
lr = 0.1
err = 1
while err > 1e-3:
    grad = gradient(params, expectation_energy)
    params = params - lr*grad
    err = abs(sum(params - last_params))
    last_params = np.array(params)
    print(err) 

2.977734310060142
0.14494446357487012
0.14747248710400607
0.03824243160315213
0.0003765805649342535


/tmp/ipykernel_10578/1088512639.py:15: ComplexWarning: Casting complex values to real discards the imaginary part
  gradients[i] = expectation_right - expectation_left


In [28]:
expectation_energy(params)

(-2.2360516529875394+0j)